In [62]:
import json
from web3 import Web3
from taxes_abi import *

In [63]:
ganache_url = 'HTTP://127.0.0.1:7545'

In [64]:
web3 = Web3(Web3.HTTPProvider(ganache_url))

In [65]:
web3.eth.defaultAccount = web3.eth.accounts[0] 
print(web3.isConnected())

True


In [66]:
contract_abi = json.loads(abi)
contract_bytecode = bytecode[0]

In [67]:
def deploy_contract(abi, bytecode):
    """
    Deploy the contract using Python and web3.py without relying on Remix
    
    Parameters
    --------------
    abi : abi of the contract
    bytecode : bytecode of the contract
    
    Returns
    --------------
    Address where the contract has been deployed
    """
    
    contract = web3.eth.contract(abi=abi, bytecode=bytecode)
    tx_hash = contract.constructor().transact()
    tx_receipt = web3.eth.waitForTransactionReceipt(tx_hash)
    return tx_receipt.contractAddress


In [68]:
contract_address = deploy_contract(abi=contract_abi, bytecode=contract_bytecode)

print(contract_address)

0xFF09AC86C4f88813e823e7c9574ecD299Ade810a


In [69]:
# Define contract
rubbish_contract = web3.eth.contract(address=contract_address, abi=contract_abi)

In [70]:
roles = ['municipality'] + [f'citizen_{i}' for i in range(1,6)] + [f'truck_{i}' for i in range(1,3)] + [f'station_{i}' for i in range(1,3)]    
address_dict = {roles[i]: web3.eth.accounts[i] for i in range(len(roles))}

In [71]:
address_dict

{'municipality': '0xbA46081a10806d57C014eF68F5B7140f2687F55a',
 'citizen_1': '0x5025f21A42fA921F3ddf57D127e89023dB357B4F',
 'citizen_2': '0x3ab7AC942EC02CefF8cb47BfaCE6fC7D6eBF2121',
 'citizen_3': '0x8ec718E7840f0880756Fec64C4B096c85D8F4323',
 'citizen_4': '0x311C1a6cFA359De22Bd7F40348aa57A6643D58a8',
 'citizen_5': '0xF21989123983792dF92F1195b69Ec860438D04dc',
 'truck_1': '0x969f3dE6bBbc0b87D4063Df5e4D237212D3E2416',
 'truck_2': '0x02e5d70E3E54B5E25A056Fce2Ae38967FDaa3d93',
 'station_1': '0xa9bEA797017d400564D8F7E047E0c7438a45Edf5',
 'station_2': '0xf37D1caFa6CCeD30e8229F93bA6f87223f1d1903'}

In [72]:
# All functions in the contract
rubbish_contract.all_functions()

[<Function _addGarbageCollectorRole(address)>,
 <Function _addMunicipalityManager(address)>,
 <Function _removeCitizenRole(address)>,
 <Function _removeGarbageCollectorRole(address)>,
 <Function _removeMunicipalityManagers(address)>,
 <Function addCitizen()>,
 <Function didIPayDeposit()>,
 <Function generateTrashBag()>,
 <Function getAmountNonRecyclableWaste()>,
 <Function getAmountRecyclableWaste()>,
 <Function getTaxesDue()>,
 <Function getTotalTaxesPaid()>,
 <Function owner()>,
 <Function payDeposit()>,
 <Function pickFromBin(address,uint256,uint8)>,
 <Function renounceOwnership()>,
 <Function transferOwnership(address)>]

Add citizens to the citizen list using function `addCitizen()`

In [73]:
for citizen in [f'citizen_{i}' for i in range(1,6)]:
    print(citizen)
    tx_hash = rubbish_contract.functions.addCitizen().transact({'from': address_dict[citizen]})

In [76]:
def get_info_from_citizen(contract, citizen):
    """
    Given a contract and a citizen, retrieve information regarding taxes paid/due and rubbish produced
    
    citizen must be a str
    
    Taxes paid -> only municiplaity manager can access this!
    """
    
    from_dict = {'from': address_dict[citizen]}
    paid_deposit = contract.functions.didIPayDeposit().call(from_dict)
    nonrec_waste_amount = contract.functions.getAmountNonRecyclableWaste().call(from_dict)
    rec_waste_amount = contract.functions.getAmountRecyclableWaste().call(from_dict)
    taxes_due = contract.functions.getTaxesDue().call(from_dict)
    #taxes_paid = contract.functions.getTotalTaxesPaid().call(from_dict)
    return paid_deposit, nonrec_waste_amount, rec_waste_amount, taxes_due, #taxes_paid

In [79]:
c1 = get_info_from_citizen(rubbish_contract, 'citizen_1')
c1

(False, 0, 0, 0)

In [12]:
import time

def handle_event(event):
    print(event)

def log_loop(event_filter, poll_interval):
    while True:
        for event in event_filter.get_new_entries():
            handle_event(event)
        time.sleep(poll_interval)

def main():
    block_filter = web3.eth.filter('latest')
    log_loop(block_filter, 2)

In [13]:
main()

b'\x94N\xd7\xf91\xaau\x89\xff\xe6\xe1m\xe4\xa1\xeb\xa1\xddU\xf2\xc7\x85\x07\xd9b\x076\xdf\xa7\xb9\xf6F('
b'\x14\xe3\x0f\xd5c\xc0)>\xb1\xa8%\r\xee\xd7\xdd\xd6\x82I\xfc\xc2k\xf1-\xa8\xb1\x9d;\x16\xfb\xb5\xe3-'


KeyboardInterrupt: 